# Preprocessing

In [179]:
import numpy as np
import pandas as pd

MAXEPOCH = 1000

In [180]:
file = open("Train.txt")

lines = file.readlines()

numClass, numFeature, datasetLen = 0, 0, 0

dataset = []

count = 0
for line in lines:
    if count == 0:
        var = line.split()
        numClass   = int(var[0])
        numFeature = int(var[1])
        datasetLen = int(var[2])
    else:
        var = line.split()
        data = []
        for i in range(numFeature):
            data.append(float(var[i]))
        data.append(int(var[numFeature]))
        dataset.append(data)
 
    count += 1

In [181]:
file = open("Test.txt")

lines = file.readlines()
test_dataset = []

for line in lines:
    var = line.split()
    data = []
    for i in range(numFeature):
        data.append(float(var[i]))
    data.append(int(var[numFeature]))
    test_dataset.append(data)

In [182]:
def test(dataset,w):
    count_accurate = 0
    for data in dataset:
        x = np.array(data)
        group = x[numFeature]
        x[numFeature] = 1
        x = np.array(x)
        x = x.reshape(numFeature+1,1)
        dot_product = np.dot(w,x)[0]
        predicted = -1
        if dot_product >= 0:
            predicted = 1
        else:
            predicted = 2

        if predicted==group:
            count_accurate += 1

    print("Accuracy :",float((count_accurate/len(dataset))*100))
    
    return float(count_accurate/len(dataset))

# Basic Perceptron Algorithm

In [183]:
np.random.seed(41)
w = np.random.uniform(-10,10,numFeature+1)

In [184]:
learning_rate = 0.025

In [185]:
t = 0

In [186]:
for i in range(MAXEPOCH):
    Y = []
    arr_dx = []
    for i in range(datasetLen):
        x = np.array(dataset[i])
        group = x[numFeature]
        x[numFeature] = 1
        x = x.reshape(numFeature+1,1)
        dot_product = np.dot(w,x)[0]
        if(group == 2 and dot_product>0):
            Y.append(x)
            arr_dx.append(1)
        elif(group ==1 and dot_product<0):
            Y.append(x)
            arr_dx.append(-1)
        else:
            pass
    
    sum = np.zeros(numFeature+1)
    
    for i in range(len(Y)):
        sum += arr_dx[i]*Y[i].transpose()[0]
    
    
    w = w - learning_rate*sum
    
    if len(Y) == 0:
        break
        
print('Final Weight : ', w)

Final Weight :  [-5.94740003  2.63876391 10.18632482]


In [187]:
test(test_dataset,w)

Accuracy : 96.0


0.96

# Reward and Punishment Algorithm

In [188]:
np.random.seed(41)
w = np.random.uniform(-10,10,numFeature+1)

In [189]:
learning_rate = 0.025

In [190]:
t = 0

In [191]:
for epoch in range(MAXEPOCH):
    count = 0
    for i in dataset:
        original = i
        x = np.array(i)
        group = x[numFeature]
        x[numFeature] = 1
        x = x.reshape(numFeature+1,1)
        dot_product = np.dot(w,x)[0]
        if dot_product<=0 and group == 1:
            count += 1
            w = w + learning_rate*np.array(original)
        elif dot_product >0 and group == 2:
            w = w - learning_rate*np.array(original)
            count += 1
        else:
            pass
    if count == 0:
        break

In [192]:
test(test_dataset,w)

Accuracy : 96.0


0.96

# Pocket Algorithm

### Pseudocode

1. Initialize Wp (pocket vector) with zero or any random number
2. Assing Wo (Weight at time = 0)  to Wp
3. Run basic perceptron learning algorithm
4. If Wt is better classifier (less amount of misclassification than previous) store Wt to Wp
5. Return Wp

In [193]:
np.random.seed(41)
wp = np.random.uniform(-10,10,numFeature+1)
w = wp

In [194]:
learning_rate = 0.025

In [195]:
t = 0

misclassification = test(dataset,w)*len(dataset)

Accuracy : 50.0


In [196]:
for i in range(MAXEPOCH):
    count = 0 
    for i in range(datasetLen):
        original = dataset[i]
        x = np.array(dataset[i])
        group = x[numFeature]
        x[numFeature] = 1
        x = x.reshape(numFeature+1,1)
        dot_product = np.dot(w,x)[0]
        if dot_product<=0 and group == 1:
            count += 1
            w = w + np.array(original)
        elif dot_product >0 and group == 2:
            w = w - np.array(original)
            count += 1
        else:
            pass
    
    if count<misclassification:
        misclassification = count
        wp = w
    
    if count == 0:
        break

In [197]:
print(wp)

[-29.32732753  10.31181641  58.53632482]


In [198]:
test(test_dataset,wp)

Accuracy : 96.0


0.96

# Kesler’s Construction

In [199]:
weights = []

for i in range(numClass):
    wi = np.random.uniform(-10,10,numFeature+1)
    weights.append(wi)

In [200]:
print(weights)

[array([-9.13061029, -7.67152594,  2.07731378]), array([-6.18138674,  3.37031435,  8.34895698])]


In [201]:
# classwise dataset extraction

df = pd.DataFrame(dataset)

In [202]:
classes = np.unique(df[numFeature])

In [203]:
class_wise_dataset = []

for i in classes:
    each_dataset = df.where(
                df[numFeature] == i).dropna()
    class_wise_dataset.append(each_dataset)

In [204]:
w = []
for each in weights:
    for e in each:
        w.append(e)

In [205]:
w = np.array(w)

In [206]:
w.shape

(6,)